Question 2: 
	
Previously, an intern had started some work but then left in a flurry due to Covid-19. All he left behind was the CSV provided with this exercise, that he had managed to extract from the database containing some usage information.
Unfortunately, he didn’t document his work, so the business is unsure what txn_type_code in that file means. However, the business believes that the transaction types contained are:
-	Internal Transfer (sender & recipient are both SuperBank customers) 
-	Interbank Transfer (recipient is not a SuperBank customer)
-	Saving Account Withdrawal (money pulled from savings account to the customer’s current account)
-	Auto Recurring Savings Account Contribution (money automatically pulled from savings account to the customer’s current account based on e.g. monthly rule)
-	Manual Savings Account Contribution (one-time contribution from a current to a savings account)
-	Phone Top-Up (airtime purchase)
-	Gift Payments (special transaction type – recipient can but does not have to be a SuperBank customer)

    1. Based on the exploratory analysis of the data, what do you think the values in txn_type_code column represent?
    2. Given the transactions (one per row) as specified in the training set, if you need to predict the churn risks (probability of churning) for every customer in this group of customers, which machine learning algorithm would you select? Explain your selection. Define your own definition of ‘churn’, as you do exploratory analysis on the data.
    3.  Use any programing language you are familiar with to implement your selected algorithm. Submit your code for evaluation. Please describe the code and explain how to use it on the data set.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import timedelta
from lifetimes import BetaGeoFitter
from lifetimes.utils import calibration_and_holdout_data
from lifetimes.utils import summary_data_from_transaction_data
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel('Txn_selected_sample.xlsx')
df.head()

,cif_deid,created_at,credit,req_amount,txn_type_code
0,fe5997e2d7ea1061a39b356f17feaa466d2a5300217b47...,2021-05-03 10:44:54.000,C,1000142,5
1,71fd0be4eeb38bc2530a9f4293b543a60686a26c28766b...,2021-06-15 20:51:10.000,D,500000,6
2,98cacffda9f9b44be10630aef4c941f25d4b7ed9e6d511...,2021-09-04 09:05:06.000,D,160000,6
3,efa050a28850ba60b0583327be5e3d7b42ce999bab235c...,2021-04-19 15:52:36.000,D,50000,6
4,01ac5b15bb4797e6b570cbcb81766810ec34fa1f43271b...,2021-04-30 20:31:22.000,C,350000,7


In [4]:
df.groupby('txn_type_code').req_amount.median()

txn_type_code
1       1000.0
2    3500000.0
3   -3750000.0
5    3000066.0
6     500000.0
7     150000.0
Name: req_amount, dtype: float64

### 1. We can see that the txn_type_code column comprises of 7 distinct figures, ranging from 1 to 7, with varied monetary value (req_amount), which logically equivalent to the 7-transaction-type mentioned in the question above. Since there is barely any clue on how to assign these label to specific type code, I can only conclude that the txn_type_code field represents the transaction types.


### 2. Predicting churn risks: BG/NBD model
- With the non-contractual business and the data I have at hand, a parametric model will be amongst the most suitable framework to apply into predicting customer churn rate. Using some complex machine learning model might pose the problem of underfitting when there is a lack of customers' information in the data (not enough data fields). After hours of researching, I come up with a probabilistic model called Beta Geometric Negative Binomial Bayes Distribution model (BG/NBD) model, which is a simplification model based on the popular Pareto/NBD model. The model will take in metrics based on the RFM framework with Recency, Frequency, Monetary and an additive metric called T with a slight modification compared to the traditional RFM analysis.   

### Prepare dataset

In [5]:
df.created_at = pd.to_datetime(df.created_at)
max_date = df.created_at.max()

## Apply BG/NBD model on the dataset

First, I will aggregate the initial data into RFM framework so that we can apply the model to out observations.

In [6]:
# determine recency, frequency, T, monetary value for each customer
df_rft = summary_data_from_transaction_data(
    transactions = df, 
    customer_id_col = "cif_deid", 
    datetime_col = "created_at", 
    monetary_value_col = "req_amount", 
    observation_period_end = max_date, 
    freq = "D")

pd.options.display.float_format = '{:,.0f}'.format
df_rft

,frequency,recency,T,monetary_value
cif_deid,,,,
0000f0fd4af68518c97b5bfecee96e52613db6378dfb207e88af1bf504e99def,2,95,221,"2,100,000"
00047b5e65aa3623786808b8adc18759d627dfb9332a41bb2be0ae59715bd756,0,0,239,0
00066ddb1845802f1cf4345a75016f5ebb865157e8debaf41507927b390115a9,0,0,234,0
000850206459a245be631fefd9d600b9c8283fbf3e07da907d9252c366545f85,5,231,261,"1,943,800"
000a28190c6b5034a466804f6cdecd2c5687946bc951bd1753697c51e76a6aa2,0,0,49,0
...,...,...,...,...
fff48e59499bd6f5b2461177728ad8aba3c7d046f87a7f51aab9d46acefc3883,3,104,233,"436,667"
fff5fe11546f390649538a42bcd787f8ae3b9f4232d0a622e38f066c58b012e5,0,0,114,0
fff82aa3273c31d738154cb236dc9dc782279cade1795f09100159545f3d0586,0,0,141,0


### Now that we have a neat rfm data that represent each user profile with the following metrics:
    + Frequency: the number of times customers make a payment request during the analyzed period
    + Recency: slightly modified, indicating the number of days between the first and the last transaction
    + Monetary: the average value of customer's transactions
    + T (longetivity): the number of days from the customer's first transaction

## Fitting the model

In [7]:
# BG/NBD model
bgf = BetaGeoFitter(penalizer_coef=1e-06)
bgf.fit(
        frequency = df_rft["frequency"], 
        recency = df_rft["recency"], 
        T = df_rft["T"],   
        weights = None,  
        verbose = True,   
        tol = 1e-06)

Optimization terminated successfully.
         Current function value: -1.535862
         Iterations: 27
         Function evaluations: 28
         Gradient evaluations: 28


<lifetimes.BetaGeoFitter: fitted with 56271 subjects, a: 0.49, alpha: 38.96, b: 6.27, r: 0.61>

### Predicting customer churn probability

Now we get into the main part, predicting the churn risk. The lifetimes package has a function that computes the probability of being alive, which can conversely be translated into the churn risks.

In [8]:
# probability that a customer is alive for each customer in dataframe
prob_alive = bgf.conditional_probability_alive(
        frequency = df_rft["frequency"], 
        recency = df_rft["recency"], 
        T = df_rft["T"])

df_rft["churn_risks"] = 1 - prob_alive
pd.options.display.float_format = '{:,.2f}'.format
df_rft.head()

,frequency,recency,T,monetary_value,churn_risks
cif_deid,,,,,
0000f0fd4af68518c97b5bfecee96e52613db6378dfb207e88af1bf504e99def,2.00,95.00,221.00,"2,100,000.00",0.28
00047b5e65aa3623786808b8adc18759d627dfb9332a41bb2be0ae59715bd756,0.00,0.00,239.00,0.00,0.00
00066ddb1845802f1cf4345a75016f5ebb865157e8debaf41507927b390115a9,0.00,0.00,234.00,0.00,0.00
000850206459a245be631fefd9d600b9c8283fbf3e07da907d9252c366545f85,5.00,231.00,261.00,"1,943,800.00",0.08
000a28190c6b5034a466804f6cdecd2c5687946bc951bd1753697c51e76a6aa2,0.00,0.00,49.00,0.00,0.00


Now that I have predicted the individual probability of churn of each customer based on the frequency and recency figures. With that information, we can query which customers are about to get away from our business and can propose some strategies to retain that customer

In [9]:
# Query a list of customers whose churn rate exceeds 80%
df_rft[df_rft.churn_risks >=0.8]

,frequency,recency,T,monetary_value,churn_risks
cif_deid,,,,,
007c067ef0d83a8c415d3c0b5065682b155de2f2377c78afc0c0334e3824ef41,16.00,117.00,254.00,"4,027,390.12",1.00
00977aad48a6565a9916e509725fe182fe7ac9a5db218c3f39721de2df5b15dd,3.00,16.00,229.00,"47,333.33",0.95
00f8f037ff742199cf95924e8b365928c289fd4140bcaf124bd64a792c738b32,2.00,10.00,227.00,"26,172.50",0.85
016c7d2b7bc6b08fc46a8de1b876b71e86651c9d142d09eeb6d75a73835604d8,2.00,6.00,240.00,"50,000.00",0.89
01952d5267cc92aa076d506a5848df69b95c92c29327ac5c1a4f0f7be84f34c8,5.00,65.00,190.00,"11,107,245.20",0.80
...,...,...,...,...,...
ff53bcd1b73edd52ad4bff61750523d2509cfeaa69aad29ae85d891ce052248e,3.00,20.00,181.00,"1,000,000.00",0.87
ff906a913a8f41566c8c5fab7ef223f65eb43dca409fba0f7405ccfd2ff104e1,2.00,5.00,240.00,"25,500.00",0.89
ffc2025cb056943cd6f3786d3b4e4552943b8d2bc3255f4855fb7c666c3ab274,5.00,77.00,218.00,"7,553,118.00",0.81
